In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from bot_factory import TaskBOT, BotAction

In [3]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
import asyncio

In [4]:
async def async_generate(chain):
    resp = await chain.arun(dummy="dummy")
    #print(resp)
    return resp

async def generate_concurrently(chains):
    tasks = [async_generate(chain) for chain in chains]
    rr = await asyncio.gather(*tasks)
    return rr

In [5]:
attributes = ["人名", "電影名", "公司名", "組織機構名稱", "地名", "活動名稱"]
bots = []
sys_msg = "你是一隻任務型機器人，負責提取使用者輸入的文字資訊。主要是提取文字中關於「{attr}」，的內容。"
extract_key = "{attr}"
extract_description = "把使用者對話內容，跟「{attr}」有關的提取出來"
_configs = [sys_msg, extract_key, extract_description]

In [6]:
## init bots
for attr in attributes:
    tmps = []
    for c in _configs:
        t = c.format(attr=attr)
        tmps.append(t)
    print(tmps[0])

    tmpbot = TaskBOT(BotAction.INFO_EXTRACT, temperature = 0.0, 
            system_message=tmps[0],
            Extract_key=tmps[1], Extract_description=tmps[2])
    bots.append(tmpbot)

你是一隻任務型機器人，負責提取使用者輸入的文字資訊。主要是提取文字中關於「人名」，的內容。
你是一隻任務型機器人，負責提取使用者輸入的文字資訊。主要是提取文字中關於「電影名」，的內容。
你是一隻任務型機器人，負責提取使用者輸入的文字資訊。主要是提取文字中關於「公司名」，的內容。
你是一隻任務型機器人，負責提取使用者輸入的文字資訊。主要是提取文字中關於「組織機構名稱」，的內容。
你是一隻任務型機器人，負責提取使用者輸入的文字資訊。主要是提取文字中關於「地名」，的內容。
你是一隻任務型機器人，負責提取使用者輸入的文字資訊。主要是提取文字中關於「活動名稱」，的內容。


In [7]:
texts = [
    "約翰是位知名的演員，他曾在《鐵達尼號》中飾演傑克。",
    "新加坡是一個充滿活力的國家，擁有繁榮的金融中心，例如瑞聲科技和新加坡航空。",
    "昨天，我在台北的國立故宮博物院參觀了一個精彩的展覽，展示了中國古代的文物。",
    "我在新竹的清華大學攻讀資訊工程學位，每天都會接觸到最新的科技和研究。",
    "昨晚，我和朋友在北京的烤鴨店品嚐了正宗的北京烤鴨。",
    "我正在閱讀一本關於人工智慧的書籍，它由斯坦福大學的安德魯·吳所撰寫。",
    "最近，我在網上購買了一台蘋果公司的新款筆記型電腦，它的性能非常強大。",
    "在上海的外灘，我看到了壯麗的黃浦江夜景，真是美不勝收。",    
]

In [8]:
for t in texts:
    print("input text:", t)
    chains = []
    for bot in bots:
        chains.append(bot.get_chains([], t) )
    res = await generate_concurrently(chains)
    for i, r in enumerate(res):
        tr = bots[i].output_parser.parse(r)
        print(tr)
    print()
    print()    

input text: 約翰是位知名的演員，他曾在《鐵達尼號》中飾演傑克。
{'人名': '約翰'}
{'電影名': '鐵達尼號'}
{'公司名': None}
{'組織機構名稱': None}
{'地名': None}
{'活動名稱': None}


input text: 新加坡是一個充滿活力的國家，擁有繁榮的金融中心，例如瑞聲科技和新加坡航空。
{'人名': None}
{'電影名': None}
{'公司名': ['瑞聲科技', '新加坡航空']}
{'組織機構名稱': ['瑞聲科技', '新加坡航空']}
{'地名': '新加坡'}
{'活動名稱': None}


input text: 昨天，我在台北的國立故宮博物院參觀了一個精彩的展覽，展示了中國古代的文物。
{'人名': '台北的國立故宮博物院'}
{'電影名': None}
{'公司名': None}
{'組織機構名稱': '國立故宮博物院'}
{'地名': '台北的國立故宮博物院'}
{'活動名稱': '國立故宮博物院參觀展覽'}


input text: 我在新竹的清華大學攻讀資訊工程學位，每天都會接觸到最新的科技和研究。
{'人名': '無'}
{'電影名': None}
{'公司名': None}
{'組織機構名稱': '清華大學'}
{'地名': '新竹的清華大學'}
{'活動名稱': None}


input text: 昨晚，我和朋友在北京的烤鴨店品嚐了正宗的北京烤鴨。
{'人名': None}
{'電影名': None}
{'公司名': None}
{'組織機構名稱': None}
{'地名': '北京'}
{'活動名稱': ''}


input text: 我正在閱讀一本關於人工智慧的書籍，它由斯坦福大學的安德魯·吳所撰寫。


Retrying langchain.chat_models.openai.acompletion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


{'人名': '安德魯·吳'}
{'電影名': None}
{'公司名': '斯坦福大學'}
{'組織機構名稱': '斯坦福大學'}
{'地名': None}
{'活動名稱': None}


input text: 最近，我在網上購買了一台蘋果公司的新款筆記型電腦，它的性能非常強大。
{'人名': None}
{'電影名': None}
{'公司名': '蘋果公司'}
{'組織機構名稱': '蘋果公司'}
{'地名': None}
{'活動名稱': None}


input text: 在上海的外灘，我看到了壯麗的黃浦江夜景，真是美不勝收。
{'人名': None}
{'電影名': None}
{'公司名': None}
{'組織機構名稱': None}
{'地名': '上海外灘、黃浦江'}
{'活動名稱': None}


